In [1]:
# import lib
import tensorflow as tf
import numpy as np ,sys, os,cv2
import matplotlib.pyplot as plt 
from sklearn.utils import shuffle
from keras.utils import np_utils
from sklearn.preprocessing import OneHotEncoder
plt.style.use('seaborn-white'); os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
np.random.seed(678); tf.set_random_seed(678);

Using TensorFlow backend.


In [2]:
# activation functions
def tf_elu(x):     return tf.nn.elu(x)
def d_tf_elu(x):   return tf.cast(tf.greater_equal(x,0),tf.float32) +  tf.exp(tf.cast(tf.less(x,0),tf.float32) * x)
def tf_iden(x):    return x
def d_tf_iden(x):  return tf.ones_like(x)
def tf_softmax(x): return tf.nn.softmax(x)

class tf_batch_norm_layer():
    
    def __init__(self,vector_shape,axis=None):
        self.moving_mean = tf.Variable(tf.zeros(shape=[1,1,1,vector_shape],dtype=tf.float32))
        self.moving_vari = tf.Variable(tf.zeros(shape=[1,1,1,vector_shape],dtype=tf.float32))
        self.axis        = (0,1,2)
        
    def feedforward(self,input,training_phase,eps = 1e-8):
        self.input = input
        self.input_size          = self.input.shape
        self.batch,self.h,self.w,self.c = self.input_size[0].value,self.input_size[1].value,self.input_size[2].value,self.input_size[3].value

        # Training Moving Average Mean         
        def training_fn():
            self.mean    = tf.reduce_mean(self.input,axis=self.axis ,keepdims=True)
            self.var     = tf.reduce_mean(tf.square(self.input-self.mean),axis=self.axis,keepdims=True)
            centered_data= (self.input - self.mean)/tf.sqrt(self.var + eps)
            
            update_variable = []
            update_variable.append(tf.assign(self.moving_mean,self.moving_mean*0.9 + 0.1 * self.mean ))
            update_variable.append(tf.assign(self.moving_vari,self.moving_vari*0.9 + 0.1 * self.var  ))
            return centered_data,update_variable
        
        # Testing Moving Average Mean        
        def  testing_fn():
            centered_data   = (self.input - self.moving_mean)/tf.sqrt(self.moving_vari + eps)
            update_variable = []
            update_variable.append(tf.assign(self.moving_mean,self.moving_mean))
            update_variable.append(tf.assign(self.moving_vari,self.moving_vari))
            return centered_data,update_variable
        
        self.output,update_variable = tf.cond(training_phase,true_fn=training_fn,false_fn=testing_fn)
        return self.output,update_variable
    
    def backprop(self,grad,eps = 1e-8):
        change_parts = 1.0 /(self.batch * self.h * self.w)
        grad_sigma   = tf.reduce_sum( grad *  (self.input-self.mean)     ,axis=self.axis,keepdims=True) * -0.5 * (self.var+eps) ** -1.5
        grad_mean    = tf.reduce_sum( grad *  (-1./tf.sqrt(self.var+eps)),axis=self.axis,keepdims=True) + grad_sigma * change_parts * 2.0 * tf.reduce_sum((self.input-self.mean),axis=self.axis,keepdims=True) * -1
        grad_x       = grad * 1/(tf.sqrt(self.var+eps)) + grad_sigma * change_parts * 2.0 * (self.input-self.mean) + grad_mean * change_parts
        return grad_x
class CNN():
    
    def __init__(self,k,inc,out,act=tf_elu,d_act=d_tf_elu):
        self.padding    = 'SAME'
        self.stride     = 1
        self.w          = tf.Variable(tf.random_normal([k,k,inc,out],stddev=0.05))
        self.m,self.v   = tf.Variable(tf.zeros_like(self.w)),tf.Variable(tf.zeros_like(self.w))
        self.act,self.d_act = act,d_act
        self.bn         = tf_batch_norm_layer(out)
        
    def getw(self): return self.w
    def feedforward(self,input,is_train,stride=1,padding='SAME'):
        self.padding= padding; self.stride = stride
        self.input  = input
        self.layer  = tf.nn.conv2d(input,self.w,strides=[1,self.stride,self.stride,1],padding=self.padding) 
        self.layerA = self.act(self.layer)
        self.layerBN, layerBN_up= self.bn.feedforward(self.layerA,is_train)
        return self.layerA,layerBN_up
    def backprop(self,gradient):
        gradient    = self.bn.backprop(gradient)
        grad_part_1 = gradient 
        grad_part_2 = self.d_act(self.layer) 
        grad_part_3 = self.input

        grad_middle = grad_part_1 * grad_part_2

        grad     = tf.nn.conv2d_backprop_filter(input = grad_part_3,filter_sizes = self.w.shape,out_backprop = grad_middle,
            strides=[1,self.stride,self.stride,1],padding=self.padding
        )/batch_size

        grad_pass = tf.nn.conv2d_backprop_input(input_sizes = self.input.shape,filter= self.w,  out_backprop = grad_middle,
            strides=[1,self.stride,self.stride,1],padding=self.padding
        )

        update_w = []
        update_w.append(tf.assign( self.m,self.m*beta1 + (1-beta1) * (grad)   ))
        update_w.append(tf.assign( self.v,self.v*beta2 + (1-beta2) * (grad ** 2)   ))
        m_hat = self.m / (1-beta1)
        v_hat = self.v / (1-beta2)
        update_w.append(tf.assign( self.w,self.w - learning_rate*( m_hat/(tf.sqrt(v_hat) + adam_e)   )))
        return grad_pass,update_w  

class tf_layer_norm_layer():
    
    def __init__(self,vector_shape):
        self.moving_mean = tf.Variable(tf.zeros(shape=[vector_shape,1,1,1],dtype=tf.float32))
        self.moving_vari = tf.Variable(tf.zeros(shape=[vector_shape,1,1,1],dtype=tf.float32))
        self.axis        = (1,2,3)
        
    def feedforward(self,input,training_phase=True,eps = 1e-8):
        self.input = input
        self.input_size          = self.input.shape
        self.batch,self.h,self.w,self.c = self.input_size[0].value,self.input_size[1].value,self.input_size[2].value,self.input_size[3].value

        # Training Moving Average Mean         
        def training_fn():
            self.mean    = tf.reduce_mean(self.input,axis=self.axis ,keepdims=True)
            self.var     = tf.reduce_mean(tf.square(self.input-self.mean),axis=self.axis,keepdims=True)
            centered_data= (self.input - self.mean)/tf.sqrt(self.var + eps)
            
            update_variable = []
            update_variable.append(tf.assign(self.moving_mean,self.moving_mean*0.9 + 0.1 * self.mean ))
            update_variable.append(tf.assign(self.moving_vari,self.moving_vari*0.9 + 0.1 * self.var  ))
            return centered_data,update_variable
        
        # Testing Moving Average Mean        
        def  testing_fn():
            centered_data   = (self.input - self.moving_mean)/tf.sqrt(self.moving_vari + eps)
            update_variable = []
            update_variable.append(tf.assign(self.moving_mean,self.moving_mean))
            update_variable.append(tf.assign(self.moving_vari,self.moving_vari))
            return centered_data,update_variable
        
        self.output,update_variable = tf.cond(training_phase,true_fn=training_fn,false_fn=testing_fn)
        return self.output,update_variable
    
    def backprop(self,grad,eps = 1e-8):
        change_parts = 1.0 /(self.h * self.w * self.c)
        grad_sigma   = tf.reduce_sum( grad *  (self.input-self.mean)     ,axis=self.axis,keepdims=True) * -0.5 * (self.var+eps) ** -1.5
        grad_mean    = tf.reduce_sum( grad *  (-1./tf.sqrt(self.var+eps)),axis=self.axis,keepdims=True) + grad_sigma * change_parts * 2.0 * tf.reduce_sum((self.input-self.mean),axis=self.axis,keepdims=True) * -1
        grad_x       = grad * 1/(tf.sqrt(self.var+eps)) + grad_sigma * change_parts * 2.0 * (self.input-self.mean) + grad_mean * change_parts
        return grad_x
class global_contrast_layer():
    
    def __init__(self,vector_shape,aimed_std=1.0):
        self.moving_mean = tf.Variable(tf.zeros(shape=(vector_shape,1),dtype=tf.float64))
        self.aimed_std   = tf.Variable(aimed_std,dtype=tf.float64)
        self.reg_lamda   = 10.0; self.EPS         = 1e-8
        
    def feedforward(self,input,training_phase):
        self.input      = input
        self.image_mean = tf.reduce_mean(self.input,-1)[:,None]
        self.centered   = self.input - self.image_mean
        self.square     = tf.square(self.centered)
        self.whole_mean = tf.reduce_mean(self.square,-1)[:,None]
        self.sqrt       = tf.sqrt(self.reg_lamda + self.whole_mean) + self.EPS
        
        def training_fn():
            gcn_data = self.aimed_std * (self.centered/self.sqrt)
            return gcn_data,tf.assign(self.moving_mean,self.moving_mean * 0.9 + 0.1 * self.image_mean)
        
        def  testing_fn():
            centered_data  = self.input - self.moving_mean
            squared_data   = tf.square(centered_data)
            whole_mean_data= tf.reduce_mean(squared_data,-1)[:,None]
            sqrt           = tf.sqrt(self.reg_lamda + whole_mean_data) + self.EPS
            gcn_data       = self.aimed_std * (centered_data/self.sqrt)
            return gcn_data, tf.assign(self.moving_mean,self.moving_mean)
        
        self.output,update_gcn_mean = tf.cond(training_phase,true_fn=training_fn,false_fn=testing_fn)
        return self.output,tf.assign(self.moving_mean,self.moving_mean)        
        
    def backprop(self,grad):
        grad_1 = grad
        grad_2 = self.aimed_std / self.sqrt
        grad_3 = -(self.centered ** 2 /self.sqrt ** 2) * (1.0/(self.sqrt-self.EPS)) / (self.input.shape[1].value)
        grad_pass = grad_1 * (grad_2 + grad_3) * (1-1./self.input.shape[1].value)
        return grad_pass
def global_contrast_normalization(image):
    XX = image

    # replacement for the loop
    X_average = np.mean(XX)
    XX = XX - X_average
    
    ss   = 1.0
    lmda = 10.
    # `su` is here the mean, instead of the sum
    contrast = np.sqrt(lmda + np.mean(XX**2)).astype(np.float64)
    
    if contrast > 1e-8:
        XX = ss * XX / contrasts
    else:
        XX = ss * XX 

    return XX

In [3]:
# load the data
epsilon = 1e-5
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train = x_train.reshape((x_train.shape[0],-1))
x_train = x_train - x_train.mean(axis=0)
x_train = x_train / np.sqrt((x_train ** 2).sum(axis=1))[:,None]
x_train = x_train.reshape((x_train.shape[0],32,32,3))

x_test = x_test.reshape((x_test.shape[0],-1))
x_test = x_test - x_test.mean(axis=0)
x_test = x_test / np.sqrt((x_test ** 2).sum(axis=1))[:,None]
x_test = x_test.reshape((x_test.shape[0],32,32,3))

x_mean = x_train.mean((0,1,2),keepdims=True)
x_std  = x_train.std ((0,1,2),keepdims=True)
x_min  = x_train.min ((0,1,2),keepdims=True)
x_max  = x_train.max ((0,1,2),keepdims=True)
x_train = (x_train-x_mean)/x_std
print(x_mean.shape,x_std.shape)

# x_mean = x_test.mean((1,2),keepdims=True)
# x_std  = x_test.std ((1,2),keepdims=True)
# x_min  = x_test.min ((1,2),keepdims=True)
# x_max  = x_test.max ((1,2),keepdims=True)
x_test  = (x_test -x_mean)/x_std
print(x_mean.shape,x_std.shape)

y_train = np_utils.to_categorical(y_train,10)
y_test  = np_utils.to_categorical(y_test, 10)
print(x_train.shape,y_train.shape,x_train.min(),x_train.max(),x_train.mean(),x_train.std())
print(x_test.shape,y_test.shape,x_test.min(),x_test.max(),x_test.mean(),x_test.std())

(1, 1, 1, 3) (1, 1, 1, 3)
(1, 1, 1, 3) (1, 1, 1, 3)
(50000, 32, 32, 3) (50000, 10) -5.30019145223961 6.308989141422824 -6.657471611030511e-15 1.000000000000072
(10000, 32, 32, 3) (10000, 10) -5.719583308785551 5.262344368561918 -0.0010486746274116258 0.9998776658660994


In [9]:
# declare hyper parameter
learning_rate = 0.0001 ; lr_decay = 0.01
beta1 = 0.9; beta2 = 0.999; adam_e = 1e-8
num_epoch     = 1000;   batch_size = 50 ; print_size = 1; channel_sizes = 64 

In [ ]:
# declare layer
avg_acc_train = 0; avg_acc_test  = 0; train_acc = [];test_acc = []
avg_lss_train = 0; avg_lss_test  = 0; train_lss = [];test_lss = []

l1   = CNN(3,3,128)
l2   = CNN(3,128,128)
l3   = CNN(3,128,128)
l4   = CNN(3,128,128)

l5   = CNN(3,128,128)
l6   = CNN(3,128,128)
l7   = CNN(3,128,128)
l8   = CNN(1,128,128)

l9   = CNN(1,128,10)
l9bn = tf_layer_norm_layer(batch_size)

In [10]:
# graph
x = tf.placeholder(shape=[batch_size,32,32,3],dtype=tf.float32)
y = tf.placeholder(shape=[batch_size,10],     dtype=tf.float32)
is_train = tf.placeholder_with_default(True,())

layer1,layer1_up   = l1.feedforward(x     ,is_train=is_train,stride=2)      
layer2,layer2_up   = l2.feedforward(layer1,is_train=is_train) 
layer3,layer3_up   = l3.feedforward(layer2,is_train=is_train)
layer4,layer4_up   = l4.feedforward(layer3,is_train=is_train)
layer5,layer5_up   = l5.feedforward(layer4,is_train=is_train)
layer6,layer6_up   = l6.feedforward(layer5,is_train=is_train)
layer7,layer7_up   = l7.feedforward(layer6,is_train=is_train)
layer8,layer8_up   = l8.feedforward(layer7,is_train=is_train,stride=2)
layer9,layer9_up   = l9.feedforward(layer8,is_train=is_train)
layer9bn,layer9bn_up = l9bn.feedforward(layer9,is_train)

final_global = tf.reduce_mean(layer9bn,[1,2])
final_soft   = tf_softmax(final_global) ; 
cost         = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=final_global,labels=y))
accuracy     = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(final_soft, 1), tf.argmax(y, 1)),tf.float32) )
gradient     = tf.tile((final_soft-y)[:,None,None,:],(1,8,8,1))/64

grad9bn = l9bn.backprop(gradient)
grad9, grad9_up = l9. backprop(grad9bn)
grad8,grad8_up  = l8. backprop(grad9)
grad7,grad7_up  = l7. backprop(grad8)
grad6,grad6_up  = l6. backprop(grad7)
grad5,grad5_up  = l5. backprop(grad6)
grad4,grad4_up  = l4. backprop(grad5)
grad3,grad3_up  = l3. backprop(grad4)
grad2,grad2_up  = l2. backprop(grad3)
grad1,grad1_up  = l1. backprop(grad2)

grad_update =  layer9bn_up+ \
               layer1_up + \
               layer2_up + \
               layer3_up + \
               layer4_up + \
               layer5_up + \
               layer6_up + \
               layer7_up + \
               layer8_up + \
               layer9_up + \
               grad9_up + grad8_up + grad7_up + \
               grad6_up + grad5_up + grad4_up + \
               grad3_up + grad2_up + grad1_up 

In [11]:
# start the session
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))
sess.run(tf.global_variables_initializer())

C:\ProgramData\Miniconda3\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [12]:
# start the training 
for iter in range(num_epoch):

    # Training Accuracy / Testing 
    for current_batch_index in range(0,len(x_train),batch_size):
        current_data  = x_train[current_batch_index:current_batch_index+batch_size].astype(np.float32)
        current_label = y_train[current_batch_index:current_batch_index+batch_size].astype(np.float32)
        sess_results  = sess.run([accuracy,cost,grad_update],feed_dict={x:current_data,y:current_label})
        sys.stdout.write('Current Iter : '+str(iter)+'/'+ str(num_epoch)  + ' batch : ' + str(current_batch_index) + '/'+ str(len(x_train)) + ' acc : '+str(sess_results[0]) + ' cost : '+str(sess_results[1])+ '\r')
        sys.stdout.flush(); 
        avg_acc_train = avg_acc_train + sess_results[0]
        avg_lss_train = avg_lss_train + sess_results[1]
    for current_batch_index in range(0,len(x_test), batch_size):
        current_data  = x_test[current_batch_index:current_batch_index+batch_size].astype(np.float32)
        current_label = y_test[current_batch_index:current_batch_index+batch_size].astype(np.float32)
        sess_results  = sess.run([accuracy,cost],feed_dict={x:current_data,y:current_label,is_train:False})
        sys.stdout.write('Current Iter : '+str(iter)+'/'+ str(num_epoch)  + ' batch : ' + str(current_batch_index) + '/'+ str(len(x_test)) + ' acc : '+str(sess_results[0]) + ' cost : '+str(sess_results[1])+ '\r')
        sys.stdout.flush(); 
        avg_acc_test = avg_acc_test + sess_results[0]
        avg_lss_test = avg_lss_test + sess_results[1]
        
    # ======================== print reset ========================  
    print("Current: "+ str(iter) + 
          "\tTrain Acc: "  + str(avg_acc_train/(len(x_train)/batch_size)) + 
          "\tTrain Cost: " + str(avg_lss_train/(len(x_train)/batch_size)) + 
          "\tTest Acc: "   + str(avg_acc_test/(len(x_test)/batch_size)) + 
          "\tTest Cost: "  + str(avg_lss_test/(len(x_test)/batch_size)) + 
          "\tLR: " + str(learning_rate) )
    avg_acc_train   = 0 ; avg_acc_test  = 0 ; avg_lss_train = 0 ; avg_lss_test  = 0
    x_train,y_train = shuffle(x_train,y_train); x_test,y_test   = shuffle(x_test,y_test)
    learning_rate   = learning_rate * (1. / (1. + lr_decay * iter)) 

Current: 0	Train Acc: 0.38694000005349516	Train Cost: 4.139618342638015	Test Acc: 0.3483000010997057	Test Cost: 1.950454073548317	LR: 0.0001
Current: 1	Train Acc: 0.3788000004589558	Train Cost: 1.8072605053186417	Test Acc: 0.36380000092089176	Test Cost: 1.889209316968918	LR: 0.0001
Current: 2	Train Acc: 0.40244000005722047	Train Cost: 1.7457849354743957	Test Acc: 0.40999999925494196	Test Cost: 1.7715975433588027	LR: 9.900990099009902e-05
Current: 3	Train Acc: 0.4610199994146824	Train Cost: 1.661325747013092	Test Acc: 0.46199999809265135	Test Cost: 1.6525365155935288	LR: 9.706853038245001e-05
Current: 4	Train Acc: 0.4840799994170666	Train Cost: 1.6114819722175597	Test Acc: 0.49799999967217445	Test Cost: 1.6225929844379425	LR: 9.424129163344661e-05
Current: 5	Train Acc: 0.5045399995148182	Train Cost: 1.5761821259260178	Test Acc: 0.5056000013649463	Test Cost: 1.592639667391777	LR: 9.061662657062174e-05
Current: 6	Train Acc: 0.518540000423789	Train Cost: 1.5516241396665573	Test Acc: 0.5330

Current: 52	Train Acc: 0.7724799981117249	Train Cost: 1.0931074494719506	Test Acc: 0.7183000013232231	Test Cost: 1.2246993511915207	LR: 1.0817659230065334e-09
Current: 53	Train Acc: 0.7762599962353707	Train Cost: 1.0872211033701897	Test Acc: 0.724100002348423	Test Cost: 1.2208607983589173	LR: 7.116881072411405e-10
Current: 54	Train Acc: 0.7785399965941906	Train Cost: 1.08285369515419	Test Acc: 0.7248999986052513	Test Cost: 1.2169594413042069	LR: 4.6515562564780425e-10
Current: 55	Train Acc: 0.7832199950814247	Train Cost: 1.0769993015527726	Test Acc: 0.7250000005960464	Test Cost: 1.2082418790459633	LR: 3.0204910756350923e-10
Current: 56	Train Acc: 0.7878999967575073	Train Cost: 1.0704977487325669	Test Acc: 0.7319000005722046	Test Cost: 1.22337097376585	LR: 1.9487039197645758e-10
Current: 57	Train Acc: 0.7894999965429306	Train Cost: 1.0669253004789352	Test Acc: 0.7349999997019768	Test Cost: 1.185340252816677	LR: 1.2491691793362665e-10
Current: 58	Train Acc: 0.7909799957275391	Train Cost:

Current: 104	Train Acc: 0.8788399952054023	Train Cost: 0.9149318726658822	Test Acc: 0.767299998998642	Test Cost: 1.1376661866903306	LR: 1.435725949321336e-22
Current: 105	Train Acc: 0.8817399960756301	Train Cost: 0.9121838268637658	Test Acc: 0.7692999976873398	Test Cost: 1.1097869661450386	LR: 7.037872300594784e-23
Current: 106	Train Acc: 0.8818999946713447	Train Cost: 0.9108635092377663	Test Acc: 0.7616999971866608	Test Cost: 1.1240911892056464	LR: 3.433108439314529e-23
Current: 107	Train Acc: 0.8824399966597557	Train Cost: 0.9084817982912063	Test Acc: 0.7705999973416329	Test Cost: 1.1324060088396073	LR: 1.666557494812878e-23
Current: 108	Train Acc: 0.8831999952793121	Train Cost: 0.9060852627754211	Test Acc: 0.7740999984741211	Test Cost: 1.1181820714473725	LR: 8.051002390400376e-24
Current: 109	Train Acc: 0.8852999960780144	Train Cost: 0.9048208851218223	Test Acc: 0.7684999966621399	Test Cost: 1.1186717388033867	LR: 3.8706742261540265e-24
Current: 110	Train Acc: 0.8856799949407578	Tra

Current: 156	Train Acc: 0.9318799969553947	Train Cost: 0.8130485332608223	Test Acc: 0.7780999997258187	Test Cost: 1.1054637613892555	LR: 2.785149928153561e-41
Current: 157	Train Acc: 0.9331399980783462	Train Cost: 0.811705747961998	Test Acc: 0.7846999981999397	Test Cost: 1.1179433834552766	LR: 1.0879491906849848e-41
Current: 158	Train Acc: 0.9331199972629547	Train Cost: 0.8105783404111863	Test Acc: 0.7844999992847442	Test Cost: 1.1123003852367401	LR: 4.233265333404609e-42
Current: 159	Train Acc: 0.9344599976539611	Train Cost: 0.8086774946451187	Test Acc: 0.7800999960303306	Test Cost: 1.1172972077131271	LR: 1.6408005168234916e-42
Current: 160	Train Acc: 0.9343799973130226	Train Cost: 0.8073791053295135	Test Acc: 0.7823999974131585	Test Cost: 1.112522087395191	LR: 6.3351371305926316e-43
Current: 161	Train Acc: 0.9357999972701073	Train Cost: 0.8059583713412285	Test Acc: 0.7796999970078469	Test Cost: 1.1177227926254272	LR: 2.436591204074089e-43
Current: 162	Train Acc: 0.9362199973464013	Tr

Current: 208	Train Acc: 0.958779999434948	Train Cost: 0.742874766588211	Test Acc: 0.7883999958634377	Test Cost: 1.1052970325946807	LR: 1.2710168818893736e-64
Current: 209	Train Acc: 0.9587599983811379	Train Cost: 0.7420132249593735	Test Acc: 0.7841999959945679	Test Cost: 1.1284270730614663	LR: 4.126678187952511e-65
Current: 210	Train Acc: 0.960499998986721	Train Cost: 0.7407854024767876	Test Acc: 0.7846999970078469	Test Cost: 1.1256780952215195	LR: 1.3354945592079327e-65
Current: 211	Train Acc: 0.9602799993157387	Train Cost: 0.7394887931346893	Test Acc: 0.7824999970197678	Test Cost: 1.1300421100854874	LR: 4.3080469651868796e-66
Current: 212	Train Acc: 0.9601399995088578	Train Cost: 0.7378568483591079	Test Acc: 0.7852999970316887	Test Cost: 1.1187224426865578	LR: 1.3852241045616976e-66
Current: 213	Train Acc: 0.960799998998642	Train Cost: 0.7364083146452903	Test Acc: 0.7866999965906143	Test Cost: 1.1157870587706566	LR: 4.439820847954159e-67
Current: 214	Train Acc: 0.9614800001382828	Tra

...

Current: 739	Train Acc: 0.9893600041270256	Train Cost: 0.7638708334565163	Test Acc: 0.7243000015616416	Test Cost: 1.2986205577850343	LR: 0.0
Current: 740	Train Acc: 0.988680004119873	Train Cost: 0.7658909472823143	Test Acc: 0.7261000001430511	Test Cost: 1.303753976225853	LR: 0.0
Current: 741	Train Acc: 0.988780003786087	Train Cost: 0.7680850886106491	Test Acc: 0.7284000009298325	Test Cost: 1.299196299314499	LR: 0.0
Current: 742	Train Acc: 0.9877000043988228	Train Cost: 0.7985237411856652	Test Acc: 0.7269000008702278	Test Cost: 1.3146477484703063	LR: 0.0
Current: 743	Train Acc: 0.9882800042629242	Train Cost: 0.7881411375403404	Test Acc: 0.726000000834465	Test Cost: 1.3100990176200866	LR: 0.0
Current: 744	Train Acc: 0.9884000036120415	Train Cost: 0.7843433157205582	Test Acc: 0.725699999332428	Test Cost: 1.3021171122789383	LR: 0.0
Current: 745	Train Acc: 0.9884000041484833	Train Cost: 0.7940348625183106	Test Acc: 0.7255000022053718	Test Cost: 1.3097743713855743	LR: 0.0
Current: 746	Train 

Current: 797	Train Acc: 0.9717800015211105	Train Cost: 0.8809884063601494	Test Acc: 0.7193000042438507	Test Cost: 1.3356166005134582	LR: 0.0
Current: 798	Train Acc: 0.9706800012588501	Train Cost: 0.8881574546098709	Test Acc: 0.7156000012159347	Test Cost: 1.3310950762033462	LR: 0.0
Current: 799	Train Acc: 0.96848000061512	Train Cost: 0.8919132612943649	Test Acc: 0.7135000023245811	Test Cost: 1.3359451580047608	LR: 0.0
Current: 800	Train Acc: 0.1878599997125566	Train Cost: 2.0788459411859512	Test Acc: 0.10969999929890037	Test Cost: 2.3032422745227814	LR: 0.0
Current: 801	Train Acc: 0.11141999957710505	Train Cost: 2.2433563435077666	Test Acc: 0.10639999942854046	Test Cost: 2.315266035795212	LR: 0.0
Current: 802	Train Acc: 0.11043999957293273	Train Cost: 2.2479053230285646	Test Acc: 0.10799999982118606	Test Cost: 2.3130355846881865	LR: 0.0
Current: 803	Train Acc: 0.12167999969422817	Train Cost: 2.2121222140789034	Test Acc: 0.11699999960139394	Test Cost: 2.300361764431	LR: 0.0
Current: 804	

Current: 855	Train Acc: 0.93377999740839	Train Cost: 0.985625617980957	Test Acc: 0.7009000039100647	Test Cost: 1.374644615650177	LR: 0.0
Current: 856	Train Acc: 0.9281199974417687	Train Cost: 1.0066205937862396	Test Acc: 0.6999000036716461	Test Cost: 1.3597677725553512	LR: 0.0
Current: 857	Train Acc: 0.9354799979925156	Train Cost: 0.9889547594189644	Test Acc: 0.6994000026583671	Test Cost: 1.347940205335617	LR: 0.0
Current: 858	Train Acc: 0.9350199974179267	Train Cost: 0.9911109127998352	Test Acc: 0.6980000019073487	Test Cost: 1.365814352631569	LR: 0.0
Current: 859	Train Acc: 0.9317599976062775	Train Cost: 0.9966086392998695	Test Acc: 0.6979000046849251	Test Cost: 1.34911783516407	LR: 0.0
Current: 860	Train Acc: 0.933459997177124	Train Cost: 0.9953994832038879	Test Acc: 0.7020000046491623	Test Cost: 1.3669432604312897	LR: 0.0
Current: 861	Train Acc: 0.9308399966359139	Train Cost: 0.9950749408602715	Test Acc: 0.6956000027060508	Test Cost: 1.365292729139328	LR: 0.0
Current: 862	Train Acc:

Current: 913	Train Acc: 0.10607999941334129	Train Cost: 2.321636337995529	Test Acc: 0.10449999952688813	Test Cost: 2.3639128041267394	LR: 0.0
Current: 914	Train Acc: 0.10699999934807419	Train Cost: 2.3225315544605256	Test Acc: 0.10659999968484044	Test Cost: 2.3608326387405394	LR: 0.0
Current: 915	Train Acc: 0.1106199994981289	Train Cost: 2.313378709554672	Test Acc: 0.10799999956041574	Test Cost: 2.3602174413204193	LR: 0.0
Current: 916	Train Acc: 0.1126999996304512	Train Cost: 2.302676277637482	Test Acc: 0.11259999971836805	Test Cost: 2.3444371807575224	LR: 0.0
Current: 917	Train Acc: 0.6466799960024655	Train Cost: 1.4706356108784675	Test Acc: 0.6474000051617622	Test Cost: 1.8336613631248475	LR: 0.0
Current: 918	Train Acc: 0.8947399964928627	Train Cost: 1.0808703778386115	Test Acc: 0.6789000043272972	Test Cost: 1.4024610275030136	LR: 0.0
Current: 919	Train Acc: 0.8972199964523315	Train Cost: 1.0710354289412498	Test Acc: 0.6902000053226948	Test Cost: 1.4207512122392654	LR: 0.0
Current: 9

Current: 971	Train Acc: 0.10057999934256077	Train Cost: 2.3533780291080473	Test Acc: 0.10079999942332506	Test Cost: 2.35209841132164	LR: 0.0
Current: 972	Train Acc: 0.10079999945685267	Train Cost: 2.3485613141059876	Test Acc: 0.10089999929070473	Test Cost: 2.347320617437363	LR: 0.0
Current: 973	Train Acc: 0.10119999939203263	Train Cost: 2.346042196035385	Test Acc: 0.10109999923035502	Test Cost: 2.34692791223526	LR: 0.0
Current: 974	Train Acc: 0.1008599994443357	Train Cost: 2.347978952884674	Test Acc: 0.10109999921172858	Test Cost: 2.3463817298412324	LR: 0.0
Current: 975	Train Acc: 0.10067999935895204	Train Cost: 2.3491919157505037	Test Acc: 0.10059999933466315	Test Cost: 2.352001966238022	LR: 0.0
Current: 976	Train Acc: 0.10171999938413501	Train Cost: 2.349166265964508	Test Acc: 0.11029999956488609	Test Cost: 2.3400408494472504	LR: 0.0
Current: 977	Train Acc: 0.16509999968484043	Train Cost: 2.197372750520706	Test Acc: 0.20719999961555005	Test Cost: 2.22197560608387	LR: 0.0
Current: 978

<img src="https://i.imgur.com/MsuYgdE.png" />